In [1]:
#autoreload
%load_ext autoreload
%autoreload 2

In [2]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

In [3]:
import warcraftlogs
from warcraftlogs.constants import TOKEN_DIR

from warcraftlogs import WarcraftLogsClient

client = WarcraftLogsClient(token_dir=TOKEN_DIR)

In [4]:
from warcraftlogs.pull import get_threat_query, find_npc_ids, identify_pull_clusters
help(get_threat_query)

Help on function get_threat_query in module warcraftlogs.pull:

get_threat_query(report_code, fight_id, **kwargs)



In [94]:
REPORT_ID = "qhZCwAV89d431zHR"
FIGHT_ID = 2


REPORT_ID = "f4RWhZgQDc1Fz72w"
FIGHT_ID = 25

REPORT_ID = "bvmLcF2ZAYXjV9ar"
FIGHT_ID = 8


In [95]:
query="""query($code:String){
reportData{
  report(code:$code){
    startTime
    endTime
    fights {
      id
      difficulty
      name
      kill
      startTime
      endTime
      fightPercentage
      dungeonPulls {
        endTime
        startTime
        enemyNPCs {
          id,
          gameID
        }
        name
    }
    }
    playerDetails(fightIDs:25)
    rankings
}
}}
"""

resp = client.query_public_api(query, {"code":REPORT_ID})

In [96]:
fight_info = resp['data']['reportData']['report']['fights'][FIGHT_ID-1]
startTime, endTime = fight_info['startTime'], fight_info['endTime']
startTime, endTime

(8800980, 9880754)

In [97]:
# given a fight, create the pull timer
threat_info = client.query_public_api(get_threat_query(REPORT_ID, FIGHT_ID))

In [98]:
len(threat_info['data']['reportData']['report']['events']['data'])

1000

In [99]:
nextPageTimestamp = threat_info['data']['reportData']['report']['events']['nextPageTimestamp']
nextPageTimestamp

9311820

In [100]:
INTERVAL_IN_MINUTES = 10
INTERVAL_IN_MILLISECONDS = INTERVAL_IN_MINUTES * 60 * 1000

In [101]:
threat_info = client.query_public_api(get_threat_query(REPORT_ID, FIGHT_ID))
nextPageTimestamp = threat_info['data']['reportData']['report']['events']['nextPageTimestamp']

counter, max_count = 0, 30
threat_info_lst = [threat_info]
while (nextPageTimestamp is not None and nextPageTimestamp < endTime) or counter < max_count:
    if nextPageTimestamp is None:
        break
    threat_info = client.query_public_api(get_threat_query(REPORT_ID, FIGHT_ID, startTime=nextPageTimestamp,
                                                            endTime=nextPageTimestamp + INTERVAL_IN_MILLISECONDS))
    print(threat_info)
    nextPageTimestamp = threat_info['data']['reportData']['report']['events']['nextPageTimestamp']
    threat_info_lst.append(threat_info)
    counter += 1
    print(f"Page {counter} of {max_count}, nextPageTimestamp: {nextPageTimestamp}")

{'data': {'reportData': {'report': {'events': {'data': [{'timestamp': 9311820, 'type': 'cast', 'sourceID': 91, 'sourceInstance': 4, 'targetID': 14, 'abilityGameID': 1, 'fight': 8, 'melee': True, 'targetMarker': 7}, {'timestamp': 9312385, 'type': 'cast', 'sourceID': 75, 'sourceInstance': 19, 'targetID': 14, 'abilityGameID': 1, 'fight': 8, 'melee': True, 'targetMarker': 7}, {'timestamp': 9313121, 'type': 'cast', 'sourceID': 75, 'sourceInstance': 19, 'targetID': 14, 'abilityGameID': 1, 'fight': 8, 'melee': True, 'targetMarker': 7}, {'timestamp': 9313891, 'type': 'cast', 'sourceID': 75, 'sourceInstance': 19, 'targetID': 14, 'abilityGameID': 1, 'fight': 8, 'melee': True, 'targetMarker': 7}, {'timestamp': 9314642, 'type': 'cast', 'sourceID': 75, 'sourceInstance': 19, 'targetID': 14, 'abilityGameID': 1, 'fight': 8, 'melee': True, 'targetMarker': 7}, {'timestamp': 9315397, 'type': 'cast', 'sourceID': 75, 'sourceInstance': 19, 'targetID': 14, 'abilityGameID': 1, 'fight': 8, 'melee': True, 'targ

In [102]:
npc_resp = client.query_public_api(find_npc_ids(REPORT_ID, FIGHT_ID))

In [103]:
id_to_name_df = pd.DataFrame(npc_resp['data']['reportData']['report']['masterData']['actors'])
id_to_name_dict = dict(zip(id_to_name_df['id'], id_to_name_df['name']))


In [104]:
def get_thread_df(threat_info_lst):
    thread_df = pd.DataFrame()
    for threat_info in threat_info_lst:
        thread_df = pd.concat([thread_df, pd.DataFrame(threat_info['data']['reportData']['report']['events']['data'])])
    return thread_df

In [105]:
thread_df = get_thread_df(threat_info_lst) #pd.DataFrame(threat_info['data']['reportData']['report']['events']['data'])
thread_df.shape

(1638, 11)

In [106]:
thread_df['sourceName'] = thread_df['sourceID'].apply(lambda x: id_to_name_dict.get(x, f"{x}_Unknown"))
thread_df['targetName'] = thread_df['targetID'].apply(lambda x: id_to_name_dict.get(x, f"{x}_Unknown"))

# prefix sourcename if sourceInstanceID is not null to indicate the source is an instance of an actor (e.g. a pet or a totem)
thread_df['sourceName'] = thread_df.apply(
    lambda row: f"{int(row['sourceInstance'])}_{row['sourceName']}" if pd.notna(row['sourceInstance']) and row['sourceInstance'] != 0 
    else row['sourceName'], axis=1
)

# normalize timestamp to seconds and start from 0 for easier plotting
thread_df['timestamp_seconds'] = (thread_df['timestamp'] - thread_df['timestamp'].min()) / 1000.0

In [107]:
player_ids = id_to_name_df.query("type=='Player'")['name'].unique().tolist()
player_ids[:5]

['Bushidozho', 'Kyotaka', 'Alfaloom', 'Amage', 'Beerkhan']

In [108]:
thread_df_for_analysis = thread_df.drop_duplicates(subset=['sourceName','targetName'])

In [109]:
df_with_clusters, cluster_stats = identify_pull_clusters(thread_df.drop_duplicates(subset=['sourceName','targetName']), gap_threshold=10) # this will add a column to the dataframe with the pull cluster id

In [110]:
# show uplimited rows
pd.set_option('display.max_rows', None)

thread_df[['timestamp_seconds', 'sourceName', 'targetName']].head(10)


,timestamp_seconds,sourceName,targetName
0,0.000,5_Cursed Rookguard,Alfaloom
1,0.416,1_Cursed Thunderer,Alfaloom
2,0.812,2_Cursed Thunderer,Alfaloom
3,0.812,6_Cursed Rookguard,Alfaloom
4,1.266,5_Cursed Rookguard,Alfaloom
5,1.505,5_Cursed Rookguard,Alfaloom
6,1.569,6_Cursed Rookguard,Alfaloom
7,2.312,6_Cursed Rookguard,Alfaloom
8,2.412,1_Cursed Thunderer,Alfaloom
9,2.442,Quartermaster Koratite,Kyotaka


In [111]:
pd.set_option('display.max_colwidth', None)  # Set to None to display full content of columns

#
df_with_clusters, cluster_stats = identify_pull_clusters(thread_df.drop_duplicates(subset=['sourceName','targetName']), gap_threshold=10)
# remove sourceName if in player_ids
df_with_clusters['sourceName'] = df_with_clusters.apply(lambda row: row['sourceName'] if row['sourceName'] not in player_ids else '', axis=1)

pull_df = df_with_clusters.\
    groupby('pull_cluster').agg({
        'timestamp_seconds': ['min', 'max', 'count'],
        'sourceName': [lambda x: len(x.unique()), lambda x: x.unique()],
    }).round(2)

In [112]:
# pd.set_option('display.max_colwidth', None)  # Set to None to display full content of columns

# #.drop_duplicates(subset=['sourceName','targetName'])
# df_with_clusters, cluster_stats = identify_pull_clusters(thread_df, gap_threshold=5)
# # remove sourceName if in player_ids
# df_with_clusters['sourceName'] = df_with_clusters.apply(lambda row: row['sourceName'] if row['sourceName'] not in player_ids else '', axis=1)

# df_with_clusters.\
#     groupby('pull_cluster').agg({
#         'timestamp_seconds': ['min', 'max', 'count'],
#         'sourceName': [lambda x: len(x.unique()), lambda x: x.unique()],
#     }).round(2)

## get events

In [113]:
query="""query($code:String){
reportData{
  report(code:$code){
    startTime
    endTime
    fights {
      id
      difficulty
      name
      kill
      startTime
      endTime
      fightPercentage
      dungeonPulls {
        endTime
        startTime
        enemyNPCs {
          id,
          gameID
        }
        name
    }
    }
    playerDetails(fightIDs:25)
    rankings
}
}}
"""

resp = client.query_public_api(query, {"code":REPORT_ID})
fight_info = resp['data']['reportData']['report']['fights'][FIGHT_ID-1]
startTime, endTime = fight_info['startTime'], fight_info['endTime']

def get_cast_query(report_code, fight_id,**kwargs):
    # If player_id is provided, filter to that specific player
    #source_filter = f"sourceID: {player_id}" if player_id is not None else ""
    # if start_time is not None:
    #     source_filter += f"startTime: {start_time}"
    source_filter = ""
    for key, value in kwargs.items():
        source_filter += f"{key}: {value}\n"
    #print(source_filter)
    query = """
      {
        reportData {
          report(code: "%s") {
            events(
              fightIDs: %d
              dataType: Casts
              limit: 1000
              %s
            ) {
              data
              nextPageTimestamp
            }
          }
        }
      }
      """ % (report_code, fight_id, source_filter)
    return query

counter, max_count = 0, 30


cast_info = client.query_public_api(get_cast_query(REPORT_ID, FIGHT_ID))
nextPageTimestamp = cast_info['data']['reportData']['report']['events']['nextPageTimestamp']
print(f"{len(cast_info['data']['reportData']['report']['events']['data'])} events")
cast_info_lst = [cast_info]

while (nextPageTimestamp is not None and nextPageTimestamp < endTime) or counter < max_count:
    if nextPageTimestamp is None:
        break
    cast_info = client.query_public_api(get_cast_query(REPORT_ID, FIGHT_ID, startTime=nextPageTimestamp,
                                                            endTime=nextPageTimestamp + INTERVAL_IN_MILLISECONDS))
    print(f"{len(cast_info['data']['reportData']['report']['events']['data'])} events")
    nextPageTimestamp = cast_info['data']['reportData']['report']['events']['nextPageTimestamp']
    cast_info_lst.append(cast_info)
    counter += 1
    print(f"Page {counter} of {max_count}, nextPageTimestamp: {nextPageTimestamp}")


cast_info_df = get_thread_df(cast_info_lst) 

1000 events
1000 events
Page 1 of 30, nextPageTimestamp: 9059362
1000 events
Page 2 of 30, nextPageTimestamp: 9178400
1000 events
Page 3 of 30, nextPageTimestamp: 9330583
1000 events
Page 4 of 30, nextPageTimestamp: 9447578
1000 events
Page 5 of 30, nextPageTimestamp: 9582105
1002 events
Page 6 of 30, nextPageTimestamp: 9693030
1000 events
Page 7 of 30, nextPageTimestamp: 9811639
651 events
Page 8 of 30, nextPageTimestamp: None


In [114]:
from warcraftlogs.query.events import fetch_events

In [115]:
friend_cast = fetch_events(
    client=client,
    report_code=REPORT_ID,
    fight_id=FIGHT_ID,
    data_type="Casts",
)

Fetching Casts events:  30%|███       | 9/30 [00:02<00:04,  4.27it/s]


In [116]:
enemy_cast = fetch_events(
    client=client,
    report_code=REPORT_ID,
    fight_id=FIGHT_ID,
    data_type="Casts",
    hostilityType="Enemies"
)

Fetching Casts events:   3%|▎         | 1/30 [00:00<00:06,  4.14it/s]


In [117]:
threat_info_df = fetch_events(
    client=client,
    report_code=REPORT_ID,
    fight_id=FIGHT_ID,
    data_type="Threat",
)

Fetching Threat events:   7%|▋         | 2/30 [00:00<00:07,  3.99it/s]


In [118]:
enemy_cast.shape, friend_cast.shape

((548, 12), (8653, 12))

In [ ]:
from warcraftlogs.ability_data_manager import AbilityDataManager
from typing import Dict

ability_data_manager = AbilityDataManager(client=client,cache_file="../data/ability_data_cache.json")

def augment_events_df(cast_info_df: pd.DataFrame, id_to_name_dict: Dict={}, 
                    ability_data_manager: AbilityDataManager=None, ability_id_to_name_dict: Dict={}) -> pd.DataFrame:
    """AbilityDataManager
    """
    # add sourceName and targetName
    if id_to_name_dict:
        cast_info_df['sourceName'] = cast_info_df['sourceID'].apply(lambda x: id_to_name_dict.get(x, f"{x}_Unknown"))
        cast_info_df['targetName'] = cast_info_df['targetID'].apply(lambda x: id_to_name_dict.get(x, f"{x}_Unknown"))

    # for cast events, add abilityName
    if ability_data_manager:
        ability_ids = cast_info_df['abilityGameID'].unique().tolist()
        ability_mapping_df = ability_data_manager.get_abilities(ability_ids)
        ability_id_to_name_dict = dict(zip(ability_mapping_df['id'], ability_mapping_df['name']))

    # for cast events, add abilityName
    if ability_id_to_name_dict:
        cast_info_df['abilityName'] = cast_info_df['abilityGameID'].apply(lambda x: ability_id_to_name_dict.get(x, f"{x}_Unknown"))

    # prefix targetName if targetInstanceID is not null to indicate the target is an instance of an actor (e.g. a pet or a totem)
    if 'targetInstance' in cast_info_df.columns:
        cast_info_df['targetName'] = cast_info_df.apply(
            lambda row: f"{row['targetName']}_{int(row['targetInstance'])}" if pd.notna(row['targetInstance']) and row['targetInstance'] != 0 
            else row['targetName'], axis=1
        )
    
    if 'sourceInstance' in cast_info_df.columns:
        # prefix sourceName if sourceInstanceID is not null to indicate the source is an instance of an actor (e.g. a pet or a totem)
        cast_info_df['sourceName'] = cast_info_df.apply(
            lambda row: f"{row['sourceName']}_{int(row['sourceInstance'])}" if pd.notna(row['sourceInstance']) and row['sourceInstance'] != 0 
            else row['sourceName'], axis=1
        )

    return cast_info_df

In [124]:
friend_cast_df = augment_events_df(friend_cast, id_to_name_dict, ability_data_manager)
enemy_cast_df = augment_events_df(enemy_cast, id_to_name_dict, ability_data_manager)

Querying abilities:   0%|          | 0/2 [00:00<?, ?it/s]

Querying abilities:   0%|          | 0/1 [00:00<?, ?it/s]

In [125]:
friend_cast_df['abilityName'].value_counts().reset_index().to_csv("../data/friend_cast_ability_name.csv", index=False)

In [126]:
threat_info_df_augmented = augment_events_df(
    threat_info_df,
    id_to_name_dict,
)

#.drop_duplicates(subset=['sourceName','targetName'])
df_with_clusters, cluster_stats = identify_pull_clusters(threat_info_df_augmented, gap_threshold=5)
# remove sourceName if in player_ids
df_with_clusters['sourceName'] = df_with_clusters.apply(lambda row: row['sourceName'] if row['sourceName'] not in player_ids else '', axis=1)

pull_df = df_with_clusters.\
    groupby('pull_cluster').agg({
        'timestamp_readable': ['min', 'max', 'count'],
        'sourceName': [lambda x: len(x.unique()), lambda x: sorted(x.unique())],
    }).round(2)

pull_df

timestamp_readable                 sourceName  \
                            min       max count <lambda_0>   
pull_cluster                                                 
0                      00:00:13  00:00:40   179         10   
1                      00:01:02  00:01:43   134          8   
2                      00:01:50  00:02:39   288         11   
3                      00:02:46  00:04:31   184         16   
4                      00:04:38  00:04:40     2          1   
5                      00:05:02  00:05:02     1          1   
6                      00:05:08  00:05:14     3          1   
7                      00:05:20  00:05:30     5          1   
8                      00:05:36  00:05:36     1          1   
9                      00:05:58  00:05:58     1          1   
10                     00:06:03  00:06:11     4          1   
11                     00:06:16  00:06:26     6          2   
12                     00:06:43  00:06:43     4          1   
13                     00:06:50  00:07:23   106          8   
14                     00:07:30  00:09:21   209         13   
15                     00:09:29  00:09:47    33         11   
16                     00:09:56  00:10:23    12          1   
17                     00:10:29  00:10:31     2          1   
18                     00:10:39  00:10:43     3          1   
19                     00:10:52  00:11:06     8          1   
20                     00:11:11  00:11:13     2          1   
21                     00:11:19  00:11:19     1          1   
22                     00:11:25  00:12:05    38          5   
23                     00:12:14  00:12:37    99         20   
24                     00:12:54  00:14:58   274         40   
25                     00:15:04  00:15:13     9          3   
26                     00:15:33  00:15:41     5          3   
27                     00:15:52  00:15:54     2          1   
28                     00:16:02  00:16:04     2          1   
29                     00:16:25  00:16:29     3          1   
30                     00:16:34  00:16:36     2          1   
31                     00:16:41  00:16:45     3          1   
32                     00:16:58  00:17:02     3          1   
33                     00:17:24  00:17:28     3          1   
34                     00:17:36  00:17:40     3          1   
35                     00:17:47  00:17:51     3          1   
36                     00:17:59  00:17:59     1          1   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              

In [ ]:
df_with_clusters, cluster_stats = identify_pull_clusters(threat_info_df_augmented.drop_duplicates(['sourceName', 'targetName']), gap_threshold=5)
# remove sourceName if in player_ids
df_with_clusters['sourceName'] = df_with_clusters.apply(lambda row: row['sourceName'] if row['sourceName'] not in player_ids else '', axis=1)

pull_df = df_with_clusters.\
    groupby('pull_cluster').agg({
        'timestamp_readable': ['min', 'max', 'count'],
        'sourceName': [lambda x: len(x.unique()), lambda x: sorted(x.unique())],
    }).round(2)

pull_df

timestamp_readable                 sourceName  \
                            min       max count <lambda_0>   
pull_cluster                                                 
0                      00:00:13  00:00:20    10          9   
1                      00:00:32  00:00:40     9          1   
2                      00:01:02  00:01:06     6          6   
3                      00:01:18  00:01:18     1          1   
4                      00:01:28  00:01:43     5          1   
5                      00:01:53  00:02:01    11         10   
6                      00:02:26  00:02:39     8          1   
7                      00:02:46  00:02:47     4          4   
8                      00:03:00  00:03:10     6          4   
9                      00:03:17  00:03:24     3          1   
10                     00:03:29  00:03:32     3          3   
11                     00:03:38  00:03:38     2          1   
12                     00:03:45  00:03:45     1          1   
13                     00:03:55  00:03:55     3          3   
14                     00:04:04  00:04:07     3          2   
15                     00:04:16  00:04:24     5          3   
16                     00:06:26  00:06:26     1          1   
17                     00:06:50  00:06:50     3          3   
18                     00:06:55  00:06:59     4          4   
19                     00:07:13  00:07:23     7          1   
20                     00:07:30  00:07:31     4          4   
21                     00:07:59  00:07:59     1          1   
22                     00:08:13  00:08:26    13         10   
23                     00:08:49  00:08:49     1          1   
24                     00:08:58  00:08:58     1          1   
25                     00:09:04  00:09:04     2          1   
26                     00:09:12  00:09:21     4          1   
27                     00:09:29  00:09:42    19         11   
28                     00:11:19  00:11:19     1          1   
29                     00:11:25  00:11:27     4          4   
30                     00:11:57  00:11:57     1          1   
31                     00:12:02  00:12:05     2          1   
32                     00:12:14  00:12:32    37         20   
33                     00:12:54  00:13:12    26         15   
34                     00:13:31  00:13:53    34         20   
35                     00:14:03  00:14:03     2          1   
36                     00:14:13  00:14:13     1          1   
37                     00:14:27  00:14:27     1          1   
38                     00:14:33  00:14:51    15          9   
39                     00:15:12  00:15:12     1          1   
40                     00:15:33  00:15:38     3          3   
41                     00:17:59  00:17:59     1          1   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 
                                                                                                                                                                                                                                                                                                                                                                                                                                                                     <lambda_1>  
pull_cluster                                                                                                                                                                                                                                                                                      

: 

In [127]:
# set max rows to display as 2000
pd.set_option('display.max_rows', 2000)

In [128]:
# get all abilities in the cast_info_df
import tqdm
# ability_data = []
# for ability_id in tqdm.tqdm(ability_ids):
#     ability_query = generate_single_ability_query(ability_id)
#     ability_info = client.query_public_api(**ability_query)
#     ability_data.append(ability_info['data']['gameData']['ability'])

In [129]:
enemy_cast_df.query("abilityGameID==1214628")

,timestamp,type,sourceID,sourceInstance,targetID,abilityGameID,fight,sourceMarker,targetInstance,targetMarker,timestamp_seconds,timestamp_readable,sourceName,targetName,abilityName
387,9560102,cast,86,3.0,-1,1214628,8,NaN,NaN,NaN,759.122,00:12:39,Consuming Voidstone_3,Environment,Unleash Darkness
468,9660880,cast,86,2.0,-1,1214628,8,NaN,NaN,NaN,859.900,00:14:19,Consuming Voidstone_2,Environment,Unleash Darkness
490,9715546,cast,86,1.0,-1,1214628,8,NaN,NaN,NaN,914.566,00:15:14,Consuming Voidstone_1,Environment,Unleash Darkness


In [130]:
enemy_cast_df[['timestamp_readable', 'sourceName', 'abilityName', 'targetName']].\
    query("abilityName=='Unleash Darkness'") #.head(10)

,timestamp_readable,sourceName,abilityName,targetName
387,00:12:39,Consuming Voidstone_3,Unleash Darkness,Environment
468,00:14:19,Consuming Voidstone_2,Unleash Darkness,Environment
490,00:15:14,Consuming Voidstone_1,Unleash Darkness,Environment


In [131]:
from warcraftlogs.data.ability_categories import ABILITY_CATEGORIES, ABILITY_TO_CATEGORY

In [132]:
friend_cast_df['abilityCategory'] = friend_cast_df['abilityName'].apply(
    lambda x: ABILITY_TO_CATEGORY.get(x, 'Unknown')
)

In [133]:
friend_cast_df[['timestamp_readable', 'sourceName', 'abilityName', 'targetName', 'abilityCategory']]

,timestamp_readable,sourceName,abilityName,targetName,abilityCategory
0,00:00:08,Alfaloom,Blessed Hammer,Environment,Unknown
1,00:00:08,Alfaloom,Divine Steed,Environment,Unknown
2,00:00:09,Alfaloom,Blessed Hammer,Environment,Unknown
3,00:00:10,Verinys,Call Pet 4,Environment,Unknown
4,00:00:10,Kyotaka,Glide,Environment,movement
...,...,...,...,...,...
8648,00:17:58,Amsterdambee,Void Blast,Environment,basic_rotation
8649,00:17:58,Alfaloom,Empyrean Hammer,Voidstone Monstrosity,Unknown
8650,00:17:59,Mashina,Final Verdict,Voidstone Monstrosity,Unknown
8651,00:17:59,Alfaloom,Empyrean Hammer,Voidstone Monstrosity,Unknown


In [134]:
enemy_cast_df[['timestamp_readable', 'sourceName', 'abilityName', 'targetName']].to_dict("records")

[{'timestamp_readable': '00:00:17',
  'sourceName': 'Cursed Thunderer_2',
  'abilityName': 'Lightning Bolt',
  'targetName': 'Environment'},
 {'timestamp_readable': '00:00:17',
  'sourceName': 'Cursed Thunderer_1',
  'abilityName': 'Lightning Bolt',
  'targetName': 'Environment'},
 {'timestamp_readable': '00:00:19',
  'sourceName': 'Quartermaster Koratite',
  'abilityName': 'Bounding Void',
  'targetName': 'Environment'},
 {'timestamp_readable': '00:00:19',
  'sourceName': 'Cursed Thunderer_2',
  'abilityName': 'Lightning Bolt',
  'targetName': 'Kyotaka'},
 {'timestamp_readable': '00:00:20',
  'sourceName': 'Cursed Thunderer_2',
  'abilityName': 'Lightning Bolt',
  'targetName': 'Environment'},
 {'timestamp_readable': '00:00:22',
  'sourceName': 'Cursed Thunderer_1',
  'abilityName': 'Lightning Bolt',
  'targetName': 'Environment'},
 {'timestamp_readable': '00:00:22',
  'sourceName': 'Quartermaster Koratite',
  'abilityName': 'Bounding Void',
  'targetName': 'Mashina'},
 {'timestamp_re

In [135]:
"""ability_mapping_df: https://wow.zamimg.com/images/wow/icons/large/inv_cosmicvoid_wave.jpg""";
#ability_mapping_df

### visualize timelines

In [136]:
import plotly.graph_objects as go
import pandas as pd
from datetime import datetime
import numpy as np

def visualize_cast_timelines(friend_cast_df, enemy_cast_df, height=800):
    """
    Create an interactive timeline visualization of friend and enemy casts
    
    Parameters:
    -----------
    friend_cast_df : pd.DataFrame
        DataFrame containing friendly casts with columns:
        [timestamp_readable, sourceName, abilityName, targetName, abilityCategory]
    enemy_cast_df : pd.DataFrame
        DataFrame containing enemy casts with columns:
        [timestamp_readable, sourceName, abilityName, targetName]
    height : int
        Height of the plot in pixels
        
    Returns:
    --------
    plotly.graph_objects.Figure
        Interactive timeline figure
    """
    
    # Create figure
    fig = go.Figure()
    
    # Color scheme
    friend_color = 'rgba(0, 128, 255, 0.7)'  # Blue
    enemy_color = 'rgba(255, 0, 0, 0.7)'     # Red
    
    # Process friendly casts
    for player in friend_cast_df['sourceName'].unique():
        player_casts = friend_cast_df[friend_cast_df['sourceName'] == player]
        
        # Create hover text
        hover_text = player_casts.apply(
            lambda x: f"Cast: {x['abilityName']}<br>"
                     f"Target: {x['targetName']}<br>"
                     f"Category: {x['abilityCategory']}<br>"
                     f"Time: {x['timestamp_readable']}", 
            axis=1
        )
        
        fig.add_trace(go.Scatter(
            x=player_casts['timestamp_readable'],
            y=[player] * len(player_casts),
            mode='markers',
            name=player,
            text=hover_text,
            hoverinfo='text',
            marker=dict(
                symbol='diamond',
                size=10,
                color=friend_color
            )
        ))
    
    # Process enemy casts
    for enemy in enemy_cast_df['sourceName'].unique():
        enemy_casts = enemy_cast_df[enemy_cast_df['sourceName'] == enemy]
        
        # Create hover text
        hover_text = enemy_casts.apply(
            lambda x: f"Cast: {x['abilityName']}<br>"
                        f"Source: {x['sourceName']}<br>"
                     f"Target: {x['targetName']}<br>"
                        f"Time: {x['timestamp_readable']}",
            axis=1
        )
        
        fig.add_trace(go.Scatter(
            x=enemy_casts['timestamp_readable'],
            y=[enemy] * len(enemy_casts),
            mode='markers',
            name=enemy,
            text=hover_text,
            hoverinfo='text',
            marker=dict(
                symbol='x',
                size=10,
                color=enemy_color
            )
        ))
    
    # Update layout
    fig.update_layout(
        title='Cast Timeline',
        xaxis_title='Time',
        yaxis_title='Characters',
        height=height,
        showlegend=True,
        hovermode='closest',
        # Add gridlines
        xaxis=dict(
            showgrid=True,
            gridwidth=1,
            gridcolor='LightGrey'
        ),
        yaxis=dict(
            showgrid=True,
            gridwidth=1,
            gridcolor='LightGrey'
        )
    )
    
    return fig

In [137]:
import plotly.graph_objects as go
import pandas as pd
from datetime import datetime

def visualize_cast_timelines(friend_cast_df, enemy_cast_df, height=800):
    """
    Create an interactive timeline visualization of friend and enemy casts with ordered time axis
    
    Parameters:
    -----------
    friend_cast_df : pd.DataFrame
        DataFrame containing friendly casts
    enemy_cast_df : pd.DataFrame
        DataFrame containing enemy casts
    height : int
        Height of the plot in pixels
    """
    # Convert timestamp_readable to datetime for proper ordering
    def convert_time(df):
        df = df.copy()
        df['datetime'] = pd.to_datetime(df['timestamp_readable'], format='%H:%M:%S')
        return df.sort_values('datetime')
    
    friend_cast_df = convert_time(friend_cast_df)
    enemy_cast_df = convert_time(enemy_cast_df)
    
    # Create figure
    fig = go.Figure()
    
    # Color scheme
    friend_color = 'rgba(0, 128, 255, 0.7)'
    enemy_color = 'rgba(255, 0, 0, 0.7)'
    
    # Process friendly casts
    for player in friend_cast_df['sourceName'].unique():
        player_casts = friend_cast_df[friend_cast_df['sourceName'] == player]
        
        hover_text = player_casts.apply(
            lambda x: f"Time: {x['timestamp_readable']}<br>"
                     f"Cast: {x['abilityName']}<br>"
                     f"Target: {x['targetName']}<br>"
                     f"Category: {x['abilityCategory']}", 
            axis=1
        )
        
        fig.add_trace(go.Scatter(
            x=player_casts['datetime'],  # Use datetime for x-axis
            y=[player] * len(player_casts),
            mode='markers',
            name=player,
            text=hover_text,
            hoverinfo='text',
            marker=dict(
                symbol='diamond',
                size=10,
                color=friend_color
            )
        ))
    
    # Process enemy casts
    for enemy in enemy_cast_df['sourceName'].unique():
        enemy_casts = enemy_cast_df[enemy_cast_df['sourceName'] == enemy]
        
        hover_text = enemy_casts.apply(
            lambda x: f"Time: {x['timestamp_readable']}<br>"
                     f"Cast: {x['abilityName']}<br>"
                     f"Target: {x['targetName']}", 
            axis=1
        )
        
        fig.add_trace(go.Scatter(
            x=enemy_casts['datetime'],  # Use datetime for x-axis
            y=[enemy] * len(enemy_casts),
            mode='markers',
            name=enemy,
            text=hover_text,
            hoverinfo='text',
            marker=dict(
                symbol='x',
                size=10,
                color=enemy_color
            )
        ))
    
    # Update layout with proper time formatting
    fig.update_layout(
        title='Cast Timeline',
        xaxis_title='Time',
        yaxis_title='Characters',
        height=height,
        showlegend=True,
        hovermode='closest',
        xaxis=dict(
            showgrid=True,
            gridwidth=1,
            gridcolor='LightGrey',
            tickformat='%H:%M:%S',  # Format tick labels as HH:MM:SS
            type='date'
        ),
        yaxis=dict(
            showgrid=True,
            gridwidth=1,
            gridcolor='LightGrey'
        )
    )
    
    return fig

In [138]:
important_friend_cast_df = friend_cast_df[friend_cast_df['abilityCategory'].isin(['defensive_cooldowns','healing_cooldowns'])]

In [139]:
friend_cast_df[['timestamp_readable', 'sourceName', 'abilityName', 'targetName', 'abilityCategory']].tail(10)

,timestamp_readable,sourceName,abilityName,targetName,abilityCategory
8643,00:17:58,Alfaloom,Empyrean Hammer,Voidstone Monstrosity,Unknown
8644,00:17:58,Alfaloom,Empyrean Hammer,Voidstone Monstrosity,Unknown
8645,00:17:58,Alfaloom,Empyrean Hammer,Voidstone Monstrosity,Unknown
8646,00:17:58,Verinys,Cobra Shot,Voidstone Monstrosity,Unknown
8647,00:17:58,Amsterdambee,Void Blast,Voidstone Monstrosity,basic_rotation
8648,00:17:58,Amsterdambee,Void Blast,Environment,basic_rotation
8649,00:17:58,Alfaloom,Empyrean Hammer,Voidstone Monstrosity,Unknown
8650,00:17:59,Mashina,Final Verdict,Voidstone Monstrosity,Unknown
8651,00:17:59,Alfaloom,Empyrean Hammer,Voidstone Monstrosity,Unknown
8652,00:17:59,Alfaloom,Consecration,Environment,Unknown


In [140]:
all_cast_df = pd.concat([enemy_cast_df[['timestamp_readable', 'sourceName', 'abilityName', 'targetName']],
           friend_cast_df[['timestamp_readable', 'sourceName', 'abilityName', 'targetName']]]).\
           sort_values(by='timestamp_readable')

In [141]:
pull_start_time = "00:12:30"
pull_end_time = "00:16:17"

In [142]:
pd.concat([friend_cast_df.query(f"timestamp_readable>='{pull_start_time}' and timestamp_readable<='{pull_end_time}' and type=='cast' and sourceName =='Kotoha'")\
    [['timestamp_readable', 'sourceName', 'abilityName', 'targetName', 'abilityCategory']],
              enemy_cast_df.query(f"timestamp_readable>='{pull_start_time}' and timestamp_readable<='{pull_end_time}'")\
     [['timestamp_readable', 'sourceName', 'abilityName', 'targetName']]]).sort_values(by='timestamp_readable')

,timestamp_readable,sourceName,abilityName,targetName,abilityCategory
379,00:12:30,Afflicted Civilian_21,Instability,Environment,NaN
380,00:12:30,Afflicted Civilian_11,Instability,Environment,NaN
381,00:12:30,Afflicted Civilian_13,Instability,Environment,NaN
382,00:12:31,Afflicted Civilian_9,Instability,Environment,NaN
383,00:12:31,Afflicted Civilian_24,Instability,Environment,NaN
384,00:12:31,Afflicted Civilian_12,Instability,Environment,NaN
385,00:12:32,Afflicted Civilian_25,Instability,Environment,NaN
386,00:12:32,Afflicted Civilian_10,Instability,Environment,NaN
387,00:12:39,Consuming Voidstone_3,Unleash Darkness,Environment,NaN
388,00:12:56,Xal'atath_10,Xal'atath's Bargain: Devour,Environment,NaN


In [143]:
# Example usage in notebook:
fig = visualize_cast_timelines(friend_cast_df.query(f"timestamp_readable>='{pull_start_time}' and timestamp_readable<='{pull_end_time}' and type=='cast'"), 
                               enemy_cast_df.query(f"timestamp_readable>='{pull_start_time}' and timestamp_readable<='{pull_end_time}' and type=='cast'"))
fig.show()

In [144]:
df_with_clusters

,timestamp,type,sourceID,sourceInstance,targetID,abilityGameID,fight,melee,targetMarker,sourceMarker,targetInstance,timestamp_seconds,timestamp_readable,sourceName,targetName,pull_cluster
0,8814401,cast,75,5.0,14,1,8,True,7.0,NaN,NaN,13.421,00:00:13,Cursed Rookguard_5,Alfaloom,0
1,8814817,cast,77,1.0,14,1,8,True,7.0,1.0,NaN,13.837,00:00:13,Cursed Thunderer_1,Alfaloom,0
2,8815213,cast,77,2.0,14,1,8,True,7.0,NaN,NaN,14.233,00:00:14,Cursed Thunderer_2,Alfaloom,0
3,8815213,cast,75,6.0,14,1,8,True,7.0,NaN,NaN,14.233,00:00:14,Cursed Rookguard_6,Alfaloom,0
4,8815667,cast,75,5.0,14,1,8,True,7.0,NaN,NaN,14.687,00:00:14,Cursed Rookguard_5,Alfaloom,0
5,8815906,cast,75,5.0,14,1,8,True,7.0,NaN,NaN,14.926,00:00:14,Cursed Rookguard_5,Alfaloom,0
6,8815970,cast,75,6.0,14,1,8,True,7.0,NaN,NaN,14.990,00:00:14,Cursed Rookguard_6,Alfaloom,0
7,8816713,cast,75,6.0,14,1,8,True,7.0,NaN,NaN,15.733,00:00:15,Cursed Rookguard_6,Alfaloom,0
8,8816813,cast,77,1.0,14,1,8,True,7.0,1.0,NaN,15.833,00:00:15,Cursed Thunderer_1,Alfaloom,0
9,8816843,cast,78,NaN,13,1,8,True,NaN,8.0,NaN,15.863,00:00:15,Quartermaster Koratite,Kyotaka,0


In [145]:
query = """
query GetDungeonRankings($zoneID: Int!) {
  worldData {
    zone(id: $zoneID) {
      name
      encounters {
        id
        name
        characterRankings(
          className: "Warrior",
          specName: "Protection",
          bracket: 17,
          includeCombatantInfo: true,
          leaderboard:LogsOnly
        )
      }
    }
  }
}
"""

variables = {
    "zoneID": 43,  # Replace with the actual dungeon zone ID
    "metric": "dps",  # Can be "dps", "hps", "playerscore", etc.
    "className": "Warrior",  # Optional
    "specName": "Protection"          # Optional
}

        # fightRankings(
        #   bracket: 17,
        # )
resp = client.query_public_api(query, variables)

In [146]:
#resp['data']['worldData']['zone']['encounters'][0]['fightRankings']['rankings']

In [147]:
resp['data']['worldData']['zone']['encounters'][1]

{'id': 12651,
 'name': 'Darkflame Cleft',
 'characterRankings': {'page': 1,
  'hasMorePages': False,
  'count': 1,
  'rankings': [{'name': 'Aweiwarrior',
    'class': 'Warrior',
    'spec': 'Protection',
    'amount': -339999559.9864315,
    'hardModeLevel': 17,
    'duration': 1859327,
    'startTime': 1743263726895,
    'report': {'code': 'djJw6gKGmnpLBabT',
     'fightID': 3,
     'startTime': 1743260578473},
    'server': {'id': 770, 'name': '白银之手', 'region': 'CN'},
    'bracketData': 17,
    'faction': 0,
    'affixes': [9, 10, 147],
    'medal': 'bronze',
    'score': 440.01356854839,
    'leaderboard': 0,
    'talents': [{'talentID': 112110, 'points': 1},
     {'talentID': 112112, 'points': 1},
     {'talentID': 112116, 'points': 1},
     {'talentID': 112149, 'points': 1},
     {'talentID': 112150, 'points': 1},
     {'talentID': 112151, 'points': 1},
     {'talentID': 112152, 'points': 1},
     {'talentID': 112153, 'points': 1},
     {'talentID': 112155, 'points': 1},
     {'ta